# Football Data Scrapping from Football Reference

[Football Reference](https://fbref.com/en/) is like a player database where you can find data for many of the leagues and tournements. Also FBRef is supported with some useful data such as Expected Goal(xG) and other advanced stats by [StatsBomb](https://statsbomb.com/).

For this project, we will collect information, news and stats of players. We start of by finding all urls of the Leagues.

We present the competition ids for the Big 5 leagues namely, the Premier League, Serie A, La Liga, Ligue 1, and Bundesliga. If you want to scrape other leagues, you have to find their competition id.

- England | Premier League: 9
- Italy | Serie A: 11
- Spain | La Liga: 12
- France | Ligue 1: 13
- Germany | Bundesliga: 20

## Helper Packages

In [1]:
# Packages
# -----------------------------------------------------------------------------

# Install
!pip install pyjsparser
!pip install js2xml
!pip install termcolor

# Base
import pandas as pd
import numpy as np

# Visualization
import plotly.express as ex

# Web Scraping
import requests
from bs4 import BeautifulSoup
import pyjsparser

# Time Sleep
import sys
import time
from datetime import datetime
from termcolor import colored

# GC
import gc

# Itertools
import itertools

# Grafikten Data Çekmek için// To Extract Data from a Chart
import re
import js2xml
from itertools import repeat    
from pprint import pprint as pp

# String Manipulation
import re 

# Configurations
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

## Finding urls of the leagues

`fbref_league_history` function gives us all historic league urls. You just need to give a competition id list and specify the first season, which in our case chose 2017.

In [2]:
# England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20

def fbref_league_history(league_id=[9, 11, 12, 13, 20], first_season=2017):
    history = []
    for i in league_id:
        comp_history_url = "https://fbref.com/en/comps/" + str(i) + "/history" 
        r=requests.get(comp_history_url)
        soup = BeautifulSoup(r.content, "html.parser")

        # Access information of the whole season from the table.
        find_seasons = soup.find_all(class_ = "left")

        all_seasons_url = []
        for k in range(0, len(find_seasons)):
            if find_seasons[k].get('data-stat') == "year_id":
                temp = "https://fbref.com" + find_seasons[k].find_all("a")[0].attrs["href"]
                year_range = temp.split("/")[-2]
                if "-" in year_range:
                    if year_range.split("-")[0].isdigit() and year_range.split("-")[1].isdigit():
                        if i == 9:
                            if int(year_range.split("-")[0]) >= (first_season):
                                all_seasons_url.append(temp)
                        else:
                            if year_range not in [f"{yr}-{yr+1}" for yr in range(1950, first_season)]:
                                all_seasons_url.append(temp)
                elif str(i) in temp and "Stats" in temp:
                    all_seasons_url.append(temp)

        history.append(all_seasons_url)

    # All histories in a single array
    history = list(itertools.chain(*history))
    
    return history





# Test for Big 5
history = fbref_league_history(league_id = [9,11,12,13,20])
history


['https://fbref.com/en/comps/9/Premier-League-Stats',
 'https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2019-2020/2019-2020-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2018-2019/2018-2019-Premier-League-Stats',
 'https://fbref.com/en/comps/9/2017-2018/2017-2018-Premier-League-Stats',
 'https://fbref.com/en/comps/11/Serie-A-Stats',
 'https://fbref.com/en/comps/11/2021-2022/2021-2022-Serie-A-Stats',
 'https://fbref.com/en/comps/11/2020-2021/2020-2021-Serie-A-Stats',
 'https://fbref.com/en/comps/11/2019-2020/2019-2020-Serie-A-Stats',
 'https://fbref.com/en/comps/11/2018-2019/2018-2019-Serie-A-Stats',
 'https://fbref.com/en/comps/11/2017-2018/2017-2018-Serie-A-Stats',
 'https://fbref.com/en/comps/12/La-Liga-Stats',
 'https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats',
 'https://fbref.com/en/comps/12/2020-2021/2020-2021-La-Liga-Stats',
 'ht

## Finding urls of the teams

We've just found all historic league urls, so we can find all historic team urls as well. When we run the function, it gives us all team urls in last season.

In [3]:
def fbref_team_url_history(league_history):
    team_season_url = []
    for league_season_url in league_history:
        r=requests.get(league_season_url)
        soup=BeautifulSoup(r.content, "html.parser")
        teams = soup.find("table").find_all("a")
        #print(teams)
        teams = list(map(lambda x: "https://fbref.com" + x["href"], teams))
        teams = Filter(teams, ["/en/squads/"])
        team_season_url.append(teams)

    # All histories in a single array
    team_season_url  = list(itertools.chain(*team_season_url))
    return team_season_url

def Filter(string, substr):
    return [str for str in string if
            any(sub in str for sub in substr)] 

# Test for Premier League Last Season
team_season_url = fbref_team_url_history(history[:1])
team_season_url

['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/5bfb9659/Leeds-United-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/e

## Finding urls of the players

Now that we have the team urls for the season, let's find player urls of the first team in the above list, Arsenal! We use these urls constantly for finding player info, stats, news etc.

In [4]:
def fbref_player_url(team_season_url):
    player_url = []
    for turl in team_season_url:
        r=requests.get(turl)
        soup=BeautifulSoup(r.content, "html.parser")
        soup.find("div", {"id":"all_stats_standard"})
        players = soup.find("tbody").find_all("a")
        players = list(map(lambda x: x["href"], players))
        players = Filter(players, ["/en/players/"])
        #print(players)
        players = NOTFilter(players, ["matchlogs"])
        player_url.append(list(map(lambda x: "https://fbref.com" + x, players)))
        time.sleep(0.01)
    player_url  = list(itertools.chain(*player_url))
    return player_url

def NOTFilter(string, substr):
    return [str for str in string if not
            any(sub in str for sub in substr)] 

# Test for Chelsea Players
player_url = fbref_player_url(team_season_url[:1])
player_url

['https://fbref.com/en/players/466fb2c5/Aaron-Ramsdale',
 'https://fbref.com/en/players/67ac5bb8/Gabriel-Dos-Santos',
 'https://fbref.com/en/players/bc7dc64d/Bukayo-Saka',
 'https://fbref.com/en/players/972aeb2a/William-Saliba',
 'https://fbref.com/en/players/79300479/Martin-Odegaard',
 'https://fbref.com/en/players/e61b8aee/Granit-Xhaka',
 'https://fbref.com/en/players/48a5a5d6/Martinelli',
 'https://fbref.com/en/players/35e413f1/Ben-White',
 'https://fbref.com/en/players/529f49ab/Thomas-Partey',
 'https://fbref.com/en/players/51cf8561/Oleksandr-Zinchenko',
 'https://fbref.com/en/players/b66315ae/Gabriel-Jesus',
 'https://fbref.com/en/players/a53649b7/Eddie-Nketiah',
 'https://fbref.com/en/players/b3af9be1/Takehiro-Tomiyasu',
 'https://fbref.com/en/players/38ceb24a/Leandro-Trossard',
 'https://fbref.com/en/players/fce2302c/Kieran-Tierney',
 'https://fbref.com/en/players/45db685d/Jorginho',
 'https://fbref.com/en/players/6412cc03/Fabio-Vieira',
 'https://fbref.com/en/players/1b4f1169/A

## Player Info

The urls of the players are important to us as we use them to scrape all data about the players that we want. Firstly, we can collect their personal info. **fbref_player_info** helps us to create a data frame with all player info. *fbref* gives us a whole lot of player stats and information. For instance, checkout [Lionel Messi's FBRef Profile](https://fbref.com/en/players/d70ce98e/Lionel-Messi).

We generate player info for the players at Arsenal.

In [18]:
def fbref_player_info(player_url):
    player_info = []
    for completed, i in enumerate(player_url):

        # PlayerId
        playerId = i.replace("https://fbref.com/en/players/", "").split("/")[0]

        # Request
        r=requests.get(i)
        soup=BeautifulSoup(r.content, "html.parser")

        # Meta
        meta = soup.find("div", {"id":"meta"})
        #print(meta)
        # Player Name
        playerName = soup.find("h1").find("span").get_text()
        
        # Nationality
        span_tags = meta.find_all("span")

        # Loop through each span tag and extract text starting with "in"
        for span in span_tags:
            birthplace = re.sub(r'\n\s*', '', span.text) # remove newlines and spaces
            match = re.search(r'^in\b', birthplace)
            if match:
                nationality = birthplace.split(",")[-1]
                break        

        # Player Photos
        try:
            photo = soup.find("div", {"class":"media-item"}).find("img").attrs["src"]
        except:
            photo = np.nan


        # Birth
        try:
            birth = meta.find("span", {"id": "necro-birth"}).get_text(strip=True)
            #soup.find("div", {"id":"meta"}).find("span", {"id":"necro-birth"})['data-birth']
        except:
            birth = np.nan

        # Height
        try:
            # Find the span tag containing the height of the player
            height_tag = meta.find('span', text=lambda t: t and 'cm' in t)

            # Extract the height from the tag
            height = int(height_tag.text.split('cm')[0])
        except:
            height = np.nan

        # Weight
        try:
            # Find the span tag containing the weight of the player
            weight_tag = meta.find('span', text=lambda t: t and 'kg' in t)

            # Extract the height from the tag
            weight = int(weight_tag.text.split('kg')[0])
        except :
            weight = np.nan


        detail = meta.find_all("p")

        # Player Full Name
        if len(Filter([detail[0].text], ["Position", "Club", "Born", "National Team", "Citizenship"])) > 0:
                playerFullName = np.nan
        else:
            playerFullName = detail[0].get_text()

        # Position & Footed
        fp = list(map(lambda x: str(x), detail))
        position = Filter(fp, ["Position"])
        footed = Filter(fp, ["Footed"])
        if len(position) > 0:
            position = position[0].split("<strong>")[1].replace("Position:</strong>","").replace("\n", "").replace("<p>", "").replace("</p>", "").replace("\xa0", "").replace("▪", "").split("<span")[0].strip()
        else:
            position = np.nan

        if len(footed) > 0:
            footed = footed[0].split("<strong>Footed:</strong>")[1].split("<span")[0].strip().replace("</p>", "").upper()
            footed = footed.split("% ")
            if len(footed) > 1:
                foot = footed[1]
                foot_ability = int(footed[0]) 
            else:
                foot = footed[0]
                foot_ability = 100
        else:
            foot = np.nan
            foot_ability = np.nan

        # International Reputation
        try:
            ir = soup.find("ul", {"id":"bling"}).find_all("a")
            ir = list(map(lambda x: x.text.strip(), ir))
            ir = '||'.join(map(str, ir))  # The variable should be separated with ||
        except:
            ir = np.nan
            
        #Social Media
        sm = Filter(list(map(lambda x: x["href"], meta.find_all("a", href = True))), ["twitter", "instagram"])
        try:
            tw = Filter(sm, ["twitter"])[0]
        except:
            tw = np.nan
        try:
            ins = Filter(sm, ["instagram"])[0]
        except:
            ins = np.nan

        # Data Frame
        temp = pd.DataFrame({
            "FBRefId":[playerId],
            "PlayerName":[playerName],
            "PlayerFullName":[playerFullName],
            "Nationality":[nationality],
            "Photo":[photo],
            "Birth":[birth],
            "Height(cm)":[height],
            "Weight(kg)":[weight],
            "Position":[position],
            "Foot":[foot],
            "FootAbility":[foot_ability],
            "InternationalReputation":[ir],
            "PlayerUrl":[i],
            "Twitter":[tw],
            "Instagram":[ins]
        })    

        temp["PlayerFullName"] = np.where(temp.PlayerFullName.isnull(), temp.PlayerName, temp.PlayerFullName)

        player_info.append(temp)

        # Print Message
        sys.stdout.write("\r{0} players have just scraped from FBRef!".format(completed+1))
        sys.stdout.flush()

        # System Sleep
        time.sleep(0.01) 

    # Write Player Info
    player_info = pd.concat(player_info)
    
    return player_info

# Test
player_info = fbref_player_info(player_url)
player_info

34 players have just scraped from FBRef!

,FBRefId,PlayerName,PlayerFullName,Nationality,Photo,Birth,Height(cm),Weight(kg),Position,Foot,FootAbility,InternationalReputation,PlayerUrl,Twitter,Instagram
0,466fb2c5,Aaron Ramsdale,Aaron Ramsdale,United Kingdom,https://fbref.com/req/202302030/images/headsho...,"May 14, 1998",195,79.0,GK,RIGHT,100.0,NaN,https://fbref.com/en/players/466fb2c5/Aaron-Ra...,https://twitter.com/AaronRamsdale98,https://instagram.com/aaronramsdale/
0,67ac5bb8,Gabriel Dos Santos,Gabriel dos Santos Magalhães,Brazil,https://fbref.com/req/202302030/images/headsho...,"December 19, 1997",190,78.0,"DF (CB, left)",LEFT,100.0,NaN,https://fbref.com/en/players/67ac5bb8/Gabriel-...,NaN,NaN
0,bc7dc64d,Bukayo Saka,Bukayo Ayoyinka T.M. Saka,United Kingdom,https://fbref.com/req/202302030/images/headsho...,"September 5, 2001",178,64.0,FW-MF (AM),LEFT,100.0,NaN,https://fbref.com/en/players/bc7dc64d/Bukayo-Saka,https://twitter.com/BukayoSaka87,https://instagram.com/bukayosaka87/
0,972aeb2a,William Saliba,William Saliba,France,https://fbref.com/req/202302030/images/headsho...,"March 24, 2001",193,76.0,"DF (CB, right)",RIGHT,100.0,NaN,https://fbref.com/en/players/972aeb2a/William-...,NaN,https://instagram.com/w.saliba4/
0,79300479,Martin Ødegaard,Martin Ødegaard,Norway,https://fbref.com/req/202302030/images/headsho...,"December 17, 1998",179,68.0,FW-MF (AM-CM-WM),LEFT,100.0,NaN,https://fbref.com/en/players/79300479/Martin-O...,NaN,https://instagram.com/odegaard.98/
0,e61b8aee,Granit Xhaka,Granit Xhaka,Switzerland,https://fbref.com/req/202302030/images/headsho...,"September 27, 1992",185,82.0,"MF (CM-DM, left)",LEFT,100.0,2x Swiss Super League Champion,https://fbref.com/en/players/e61b8aee/Granit-X...,NaN,https://instagram.com/granitxhaka/
0,48a5a5d6,Martinelli,Gabriel Teodoro Martinelli Silva,Brazil,https://fbref.com/req/202302030/images/headsho...,"June 18, 2001",180,74.0,FW-MF,RIGHT,100.0,NaN,https://fbref.com/en/players/48a5a5d6/Martinelli,NaN,https://instagram.com/ga_martinelli01/
0,35e413f1,Ben White,Ben White,United Kingdom,https://fbref.com/req/202302030/images/headsho...,"October 8, 1997",182,73.0,"DF (CB, right)",RIGHT,100.0,NaN,https://fbref.com/en/players/35e413f1/Ben-White,https://twitter.com/ben6white,https://instagram.com/ben_white6/
0,529f49ab,Thomas Partey,Thomas Teye Partey,Ghana,https://fbref.com/req/202302030/images/headsho...,"June 13, 1993",185,78.0,MF (CM-DM),RIGHT,100.0,2x Ghana Player of the Year,https://fbref.com/en/players/529f49ab/Thomas-P...,https://twitter.com/Thomaspartey22,https://instagram.com/thomaspartey5/
0,51cf8561,Oleksandr Zinchenko,Зінченко Олександр Володимирович,Ukraine,https://fbref.com/req/202302030/images/headsho...,"December 15, 1996",175,60.0,"DF-MF (FB, left)",LEFT,100.0,4x Premier League Champion,https://fbref.com/en/players/51cf8561/Oleksand...,NaN,https://instagram.com/zinchenko_96/


## Player Performance Stats

One of the most important part in notebook is scraping of the players. The players are able to play different competitions like domestic leagues, domestic cups, international cups and national team. We can collect all data with the following code. We will have 4 different dataframes at the end of the process. You can checkout Lionel Messi's Stats [here](https://fbref.com/en/players/d70ce98e/Lionel-Messi).

In [21]:
def get_time(cond):
    if cond == "start":
        p = "This process started at "
    elif cond == "end":
        p = "The process completed at "
    print("") 
    print(colored(p + str(datetime.now().strftime("%H:%M:%S")), "green", "on_white", attrs=["bold",'reverse', 'blink']))

# Domestic League | Domestic Cups | International Cups |National Team
stats_dict = {}
for i in ["dom_lg", "dom_cup", "intl_cup", "nat_tm"]:
    stats_list = ["div_stats_standard_", "div_stats_shooting_","div_stats_passing_", "div_stats_passing_types_","div_stats_gca_", "div_stats_defense_", "div_stats_possession_","div_stats_playing_time_", "div_stats_misc_"]
    stats_list = list(map(lambda x: str(x)+i, stats_list))
    stats_dict[i] = stats_list
del i, stats_list


def get_stats(soup, purl, id_, first_season = 2017, final_season = 2021):
    
    # Player ID
    playerid = purl.replace("https://fbref.com/en/players/", "").split("/")[0]
    
    # Get Table
    temp = pd.read_html(str(soup.find("div", {"id":id_}).find("table")))[0]
        
    # Select Correct Rows
    seasons = list(map(lambda x: str(x)+"-"+str(x+1), np.arange(first_season, final_season+1, 1)))
    seasons = seasons + list(map(lambda x: str(x), np.arange(first_season,final_season+1,1)))
    temp = temp[temp["Unnamed: 0_level_0"].Season.isin(seasons)]
    

    # Rename Columns
    temp.columns = temp.columns.map('_'.join).str.strip("_")
    rename_cols = lambda x,y: str(x).replace("_level_0", "").replace("Unnamed: ", "").replace(" ", "_").replace(str(y)+"_", "")
    temp.columns = list(map(rename_cols, temp.columns, np.arange(0,40,1)))

    # Drop
    temp.drop("Matches", axis = 1, inplace = True)
    
     # Reduce Memory
    temp["Age"] = temp.Age.astype("int8")
    
    # Player ID
    temp["FBRefId"] = playerid  

    # Set Index
    if "nat_tm" in id_ or "intl_cup" in id_ :
        temp.set_index(["FBRefId", "Season", "Squad", "Comp", "Age", "LgRank"], inplace = True)
    else:
        temp["Country"] = temp["Country"].str.split(" ", expand = True)[1]
        temp.set_index(["FBRefId","Season", "Squad", "Comp", "Age", "Country", "LgRank"], inplace = True)
        
    # Rename
    prefix = id_.replace("div_stats_", "").replace("dom_lg", "").replace("dom_cup", "").replace("intl_cup", "").replace("nat_tm", "").upper()
    temp.columns = list(map(lambda x: prefix+str(x), temp.columns.tolist()))
        
        
    # Reduce Memory
    temp = temp.astype("float")
    red_cols = temp.select_dtypes("float").columns
    temp[red_cols] = temp[red_cols].astype("float16")

    return temp



def fbref_player_stats(playerid):
    # Start the process and display time
    ################################################################
    get_time("start")

    # Create Empty Lists for stats
    ################################################################
    dom_lg_df = []
    dom_cup_df = []
    intl_cup_df = []
    nat_tm_df = []

    # Start a loop for every player
    ################################################################
    for completed, select_player in enumerate(playerid):  

        # Rewrite url
        select_player = 'https://fbref.com/en/players/' + select_player + '/all_comps/'

        # Request
        r=requests.get(select_player)
        soup=BeautifulSoup(r.content, "html.parser")

        # Create Empty List As Temporary
        dom_lg_df_temp = []
        dom_cup_df_temp = []
        intl_cup_df_temp = []
        nat_tm_df_temp = []

        # Domestic League | Domestic Cups | International Cups |National Team
        for key in stats_dict.keys():
            temp = []
            for i in range(0,9):
                x = stats_dict[key][i]
                try: 
                    x = get_stats(soup, select_player, id_ = x)
                    temp.append(x)
                except:
                    pass

            try:
                if key == 'dom_lg':
                    dom_lg_df_temp.append(pd.concat(temp, axis = 1))
                elif key == 'dom_cup':
                    dom_cup_df_temp.append(pd.concat(temp, axis = 1))
                elif key =='intl_cup':
                    intl_cup_df_temp.append(pd.concat(temp, axis = 1))
                elif key =='nat_tm':
                    nat_tm_df_temp.append(pd.concat(temp, axis = 1))
            except: 
                pass

        try: 
            # Return Dataframe
            dom_lg_df.append(pd.concat(dom_lg_df_temp).drop_duplicates())
            dom_cup_df.append(pd.concat(dom_cup_df_temp).drop_duplicates())
            intl_cup_df.append(pd.concat(intl_cup_df_temp).drop_duplicates())
            nat_tm_df.append(pd.concat(nat_tm_df_temp).drop_duplicates())
        except:
            pass


        # Print Message: How many players have scraped?
        sys.stdout.write("\r{0} players have just scraped from FBRef!".format(completed+1))
        sys.stdout.flush()

        # Sleep the system
        time.sleep(0.001)


    # Reduce Memory
    ################################################################
    del key, temp, i, dom_lg_df_temp, dom_cup_df_temp, intl_cup_df_temp, nat_tm_df_temp
    gc.collect()

    # Finish the process and display time
    ################################################################
    get_time("end")


    # Transorm Data Frame
    ################################################################
    dom_lg_df2 = pd.concat(dom_lg_df).reset_index()
    dom_cup_df2 = pd.concat(dom_cup_df).reset_index()
    intl_cup_df2 = pd.concat(intl_cup_df).reset_index()
    nat_tm_df2 = pd.concat(nat_tm_df).reset_index()
    print(dom_lg_df2.shape, dom_cup_df2.shape, intl_cup_df2.shape, nat_tm_df2.shape)
    print(dom_lg_df2.FBRefId.nunique(), dom_cup_df2.FBRefId.nunique(), intl_cup_df2.FBRefId.nunique(), nat_tm_df2.FBRefId.nunique())

    # Write Data
    ################################################################
    dom_lg_df2.to_csv("performance_dom_lg.csv", index = None)
    dom_cup_df2.to_csv("performance_dom_cup.csv", index = None)
    intl_cup_df2.to_csv("performance_intl_cup.csv", index = None)
    nat_tm_df2.to_csv("performance_nat_tm.csv", index = None)
    
    return dom_lg_df2, dom_cup_df2, intl_cup_df2, nat_tm_df2
    
    
# Test
dom_lg, dom_cup, intl_cup, nat_tm = fbref_player_stats(player_info.FBRefId[:1])


This process started at 13:45:56
1 players have just scraped from FBRef!
The process completed at 13:46:39
(7, 172) (7, 152) (0, 171) (2, 151)
1 1 0 1


### Domestic League Stats

In [22]:
dom_lg

,FBRefId,Season,Squad,Comp,Age,Country,LgRank,STANDARD_MP,STANDARD_Playing_Time_Starts,STANDARD_Playing_Time_Min,STANDARD_Playing_Time_90s,STANDARD_Performance_Gls,STANDARD_Performance_Ast,STANDARD_Performance_G+A,STANDARD_Performance_G-PK,STANDARD_Performance_PK,STANDARD_Performance_PKatt,STANDARD_Performance_CrdY,STANDARD_Performance_CrdR,STANDARD_Expected_xG,STANDARD_Expected_npxG,STANDARD_Expected_xAG,STANDARD_Expected_npxG+xAG,STANDARD_Progression_PrgC,STANDARD_Progression_PrgP,STANDARD_Progression_PrgR,STANDARD_Per_90_Minutes_Gls,STANDARD_Per_90_Minutes_Ast,STANDARD_Per_90_Minutes_G+A,STANDARD_Per_90_Minutes_G-PK,STANDARD_Per_90_Minutes_G+A-PK,STANDARD_Per_90_Minutes_xG,STANDARD_Per_90_Minutes_xAG,STANDARD_Per_90_Minutes_xG+xAG,STANDARD_Per_90_Minutes_npxG,STANDARD_Per_90_Minutes_npxG+xAG,PASSING_90s,PASSING_Total_Cmp,PASSING_Total_Att,PASSING_Total_Cmp%,PASSING_Total_TotDist,PASSING_Total_PrgDist,PASSING_Short_Cmp,PASSING_Short_Att,PASSING_Short_Cmp%,PASSING_Medium_Cmp,PASSING_Medium_Att,PASSING_Medium_Cmp%,PASSING_Long_Cmp,PASSING_Long_Att,PASSING_Long_Cmp%,PASSING_Ast,PASSING_xAG,PASSING_xA,PASSING_A-xAG,PASSING_KP,PASSING_1/3,PASSING_PPA,PASSING_CrsPA,PASSING_PrgP,PASSING_TYPES_90s,PASSING_TYPES_Att,PASSING_TYPES_Pass_Types_Live,PASSING_TYPES_Pass_Types_Dead,PASSING_TYPES_Pass_Types_FK,PASSING_TYPES_Pass_Types_TB,PASSING_TYPES_Pass_Types_Sw,PASSING_TYPES_Pass_Types_Crs,PASSING_TYPES_Pass_Types_TI,PASSING_TYPES_Pass_Types_CK,PASSING_TYPES_Corner_Kicks_In,PASSING_TYPES_Corner_Kicks_Out,PASSING_TYPES_Corner_Kicks_Str,PASSING_TYPES_Outcomes_Cmp,PASSING_TYPES_Outcomes_Off,PASSING_TYPES_Outcomes_Blocks,GCA_90s,GCA_SCA_SCA,GCA_SCA_SCA90,GCA_SCA_Types_PassLive,GCA_SCA_Types_PassDead,GCA_SCA_Types_TO,GCA_SCA_Types_Sh,GCA_SCA_Types_Fld,GCA_SCA_Types_Def,GCA_GCA_GCA,GCA_GCA_GCA90,GCA_GCA_Types_PassLive,GCA_GCA_Types_PassDead,GCA_GCA_Types_TO,GCA_GCA_Types_Sh,GCA_GCA_Types_Fld,GCA_GCA_Types_Def,DEFENSE_90s,DEFENSE_Tackles_Tkl,DEFENSE_Tackles_TklW,DEFENSE_Tackles_Def_3rd,DEFENSE_Tackles_Mid_3rd,DEFENSE_Tackles_Att_3rd,DEFENSE_Challenges_Tkl,DEFENSE_Challenges_Att,DEFENSE_Challenges_Tkl%,DEFENSE_Challenges_Lost,DEFENSE_Blocks_Blocks,DEFENSE_Blocks_Sh,DEFENSE_Blocks_Pass,DEFENSE_Int,DEFENSE_Tkl+Int,DEFENSE_Clr,DEFENSE_Err,POSSESSION_90s,POSSESSION_Touches_Touches,POSSESSION_Touches_Def_Pen,POSSESSION_Touches_Def_3rd,POSSESSION_Touches_Mid_3rd,POSSESSION_Touches_Att_3rd,POSSESSION_Touches_Att_Pen,POSSESSION_Touches_Live,POSSESSION_Take-Ons_Att,POSSESSION_Take-Ons_Succ,POSSESSION_Take-Ons_Succ%,POSSESSION_Take-Ons_Tkld,POSSESSION_Take-Ons_Tkld%,POSSESSION_Carries_Carries,POSSESSION_Carries_TotDist,POSSESSION_Carries_PrgDist,POSSESSION_Carries_PrgC,POSSESSION_Carries_1/3,POSSESSION_Carries_CPA,POSSESSION_Carries_Mis,POSSESSION_Carries_Dis,POSSESSION_Receiving_Rec,POSSESSION_Receiving_PrgR,PLAYING_TIME_MP,PLAYING_TIME_Playing_Time_Min,PLAYING_TIME_Playing_Time_Mn/MP,PLAYING_TIME_Playing_Time_Min%,PLAYING_TIME_Playing_Time_90s,PLAYING_TIME_Starts_Starts,PLAYING_TIME_Starts_Mn/Start,PLAYING_TIME_Starts_Compl,PLAYING_TIME_Subs_Subs,PLAYING_TIME_Subs_Mn/Sub,PLAYING_TIME_Subs_unSub,PLAYING_TIME_Team_Success_PPM,PLAYING_TIME_Team_Success_onG,PLAYING_TIME_Team_Success_onGA,PLAYING_TIME_Team_Success_+/-,PLAYING_TIME_Team_Success_+/-90,PLAYING_TIME_Team_Success_On-Off,PLAYING_TIME_Team_Success_(xG)_onxG,PLAYING_TIME_Team_Success_(xG)_onxGA,PLAYING_TIME_Team_Success_(xG)_xG+/-,PLAYING_TIME_Team_Success_(xG)_xG+/-90,PLAYING_TIME_Team_Success_(xG)_On-Off,MISC_90s,MISC_Performance_CrdY,MISC_Performance_CrdR,MISC_Performance_2CrdY,MISC_Performance_Fls,MISC_Performance_Fld,MISC_Performance_Off,MISC_Performance_Crs,MISC_Performance_Int,MISC_Performance_TklW,MISC_Performance_PKwon,MISC_Performance_PKcon,MISC_Performance_OG,MISC_Performance_Recov,MISC_Aerial_Duels_Won,MISC_Aerial_Duels_Lost,MISC_Aerial_Duels_Won%
0,466fb2c5,2017-2018,Bournemouth,1. Premier League,19,ENG,12th,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

### Domestic Cups Stats

In [23]:
dom_cup

,FBRefId,Season,Squad,Comp,Age,Country,LgRank,STANDARD_MP,STANDARD_Playing_Time_Starts,STANDARD_Playing_Time_Min,STANDARD_Playing_Time_90s,STANDARD_Performance_Gls,STANDARD_Performance_Ast,STANDARD_Performance_G+A,STANDARD_Performance_G-PK,STANDARD_Performance_PK,STANDARD_Performance_PKatt,STANDARD_Performance_CrdY,STANDARD_Performance_CrdR,STANDARD_Progression_PrgC,STANDARD_Progression_PrgP,STANDARD_Progression_PrgR,STANDARD_Per_90_Minutes_Gls,STANDARD_Per_90_Minutes_Ast,STANDARD_Per_90_Minutes_G+A,STANDARD_Per_90_Minutes_G-PK,STANDARD_Per_90_Minutes_G+A-PK,PASSING_90s,PASSING_Total_Cmp,PASSING_Total_Att,PASSING_Total_Cmp%,PASSING_Total_TotDist,PASSING_Total_PrgDist,PASSING_Short_Cmp,PASSING_Short_Att,PASSING_Short_Cmp%,PASSING_Medium_Cmp,PASSING_Medium_Att,PASSING_Medium_Cmp%,PASSING_Long_Cmp,PASSING_Long_Att,PASSING_Long_Cmp%,PASSING_Ast,PASSING_A-xAG,PASSING_KP,PASSING_1/3,PASSING_PPA,PASSING_CrsPA,PASSING_PrgP,PASSING_TYPES_90s,PASSING_TYPES_Att,PASSING_TYPES_Pass_Types_Live,PASSING_TYPES_Pass_Types_Dead,PASSING_TYPES_Pass_Types_FK,PASSING_TYPES_Pass_Types_TB,PASSING_TYPES_Pass_Types_Sw,PASSING_TYPES_Pass_Types_Crs,PASSING_TYPES_Pass_Types_TI,PASSING_TYPES_Pass_Types_CK,PASSING_TYPES_Corner_Kicks_In,PASSING_TYPES_Corner_Kicks_Out,PASSING_TYPES_Corner_Kicks_Str,PASSING_TYPES_Outcomes_Cmp,PASSING_TYPES_Outcomes_Off,PASSING_TYPES_Outcomes_Blocks,GCA_90s,GCA_SCA_SCA,GCA_SCA_SCA90,GCA_SCA_Types_PassLive,GCA_SCA_Types_PassDead,GCA_SCA_Types_TO,GCA_SCA_Types_Sh,GCA_SCA_Types_Fld,GCA_SCA_Types_Def,GCA_GCA_GCA,GCA_GCA_GCA90,GCA_GCA_Types_PassLive,GCA_GCA_Types_PassDead,GCA_GCA_Types_TO,GCA_GCA_Types_Sh,GCA_GCA_Types_Fld,GCA_GCA_Types_Def,DEFENSE_90s,DEFENSE_Tackles_Tkl,DEFENSE_Tackles_TklW,DEFENSE_Tackles_Def_3rd,DEFENSE_Tackles_Mid_3rd,DEFENSE_Tackles_Att_3rd,DEFENSE_Challenges_Tkl,DEFENSE_Challenges_Att,DEFENSE_Challenges_Tkl%,DEFENSE_Challenges_Lost,DEFENSE_Blocks_Blocks,DEFENSE_Blocks_Sh,DEFENSE_Blocks_Pass,DEFENSE_Int,DEFENSE_Tkl+Int,DEFENSE_Clr,DEFENSE_Err,POSSESSION_90s,POSSESSION_Touches_Touches,POSSESSION_Touches_Def_Pen,POSSESSION_Touches_Def_3rd,POSSESSION_Touches_Mid_3rd,POSSESSION_Touches_Att_3rd,POSSESSION_Touches_Att_Pen,POSSESSION_Touches_Live,POSSESSION_Take-Ons_Att,POSSESSION_Take-Ons_Succ,POSSESSION_Take-Ons_Succ%,POSSESSION_Take-Ons_Tkld,POSSESSION_Take-Ons_Tkld%,POSSESSION_Carries_Carries,POSSESSION_Carries_TotDist,POSSESSION_Carries_PrgDist,POSSESSION_Carries_PrgC,POSSESSION_Carries_1/3,POSSESSION_Carries_CPA,POSSESSION_Carries_Mis,POSSESSION_Carries_Dis,POSSESSION_Receiving_Rec,POSSESSION_Receiving_PrgR,PLAYING_TIME_MP,PLAYING_TIME_Playing_Time_Min,PLAYING_TIME_Playing_Time_Mn/MP,PLAYING_TIME_Playing_Time_Min%,PLAYING_TIME_Playing_Time_90s,PLAYING_TIME_Starts_Starts,PLAYING_TIME_Starts_Mn/Start,PLAYING_TIME_Starts_Compl,PLAYING_TIME_Subs_Subs,PLAYING_TIME_Subs_Mn/Sub,PLAYING_TIME_Subs_unSub,PLAYING_TIME_Team_Success_PPM,PLAYING_TIME_Team_Success_onG,PLAYING_TIME_Team_Success_onGA,PLAYING_TIME_Team_Success_+/-,PLAYING_TIME_Team_Success_+/-90,PLAYING_TIME_Team_Success_On-Off,MISC_90s,MISC_Performance_CrdY,MISC_Performance_CrdR,MISC_Performance_2CrdY,MISC_Performance_Fls,MISC_Performance_Fld,MISC_Performance_Off,MISC_Performance_Crs,MISC_Performance_Int,MISC_Performance_TklW,MISC_Performance_PKwon,MISC_Performance_PKcon,MISC_Performance_OG
0,466fb2c5,2017-2018,Bournemouth,EFL Cup,19,ENG,5th,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,466fb2c5,2018-2019,AFC Wimbledon,FA Cup,20,ENG,9th

### International Cups Stats

In [24]:
intl_cup

,FBRefId,Season,Squad,Comp,Age,LgRank,STANDARD_MP,STANDARD_Playing_Time_Starts,STANDARD_Playing_Time_Min,STANDARD_Playing_Time_90s,STANDARD_Performance_Gls,STANDARD_Performance_Ast,STANDARD_Performance_G+A,STANDARD_Performance_G-PK,STANDARD_Performance_PK,STANDARD_Performance_PKatt,STANDARD_Performance_CrdY,STANDARD_Performance_CrdR,STANDARD_Expected_xG,STANDARD_Expected_npxG,STANDARD_Expected_xAG,STANDARD_Expected_npxG+xAG,STANDARD_Progression_PrgC,STANDARD_Progression_PrgP,STANDARD_Progression_PrgR,STANDARD_Per_90_Minutes_Gls,STANDARD_Per_90_Minutes_Ast,STANDARD_Per_90_Minutes_G+A,STANDARD_Per_90_Minutes_G-PK,STANDARD_Per_90_Minutes_G+A-PK,STANDARD_Per_90_Minutes_xG,STANDARD_Per_90_Minutes_xAG,STANDARD_Per_90_Minutes_xG+xAG,STANDARD_Per_90_Minutes_npxG,STANDARD_Per_90_Minutes_npxG+xAG,PASSING_90s,PASSING_Total_Cmp,PASSING_Total_Att,PASSING_Total_Cmp%,PASSING_Total_TotDist,PASSING_Total_PrgDist,PASSING_Short_Cmp,PASSING_Short_Att,PASSING_Short_Cmp%,PASSING_Medium_Cmp,PASSING_Medium_Att,PASSING_Medium_Cmp%,PASSING_Long_Cmp,PASSING_Long_Att,PASSING_Long_Cmp%,PASSING_Ast,PASSING_xAG,PASSING_xA,PASSING_A-xAG,PASSING_KP,PASSING_1/3,PASSING_PPA,PASSING_CrsPA,PASSING_PrgP,PASSING_TYPES_90s,PASSING_TYPES_Att,PASSING_TYPES_Pass_Types_Live,PASSING_TYPES_Pass_Types_Dead,PASSING_TYPES_Pass_Types_FK,PASSING_TYPES_Pass_Types_TB,PASSING_TYPES_Pass_Types_Sw,PASSING_TYPES_Pass_Types_Crs,PASSING_TYPES_Pass_Types_TI,PASSING_TYPES_Pass_Types_CK,PASSING_TYPES_Corner_Kicks_In,PASSING_TYPES_Corner_Kicks_Out,PASSING_TYPES_Corner_Kicks_Str,PASSING_TYPES_Outcomes_Cmp,PASSING_TYPES_Outcomes_Off,PASSING_TYPES_Outcomes_Blocks,GCA_90s,GCA_SCA_SCA,GCA_SCA_SCA90,GCA_SCA_Types_PassLive,GCA_SCA_Types_PassDead,GCA_SCA_Types_TO,GCA_SCA_Types_Sh,GCA_SCA_Types_Fld,GCA_SCA_Types_Def,GCA_GCA_GCA,GCA_GCA_GCA90,GCA_GCA_Types_PassLive,GCA_GCA_Types_PassDead,GCA_GCA_Types_TO,GCA_GCA_Types_Sh,GCA_GCA_Types_Fld,GCA_GCA_Types_Def,DEFENSE_90s,DEFENSE_Tackles_Tkl,DEFENSE_Tackles_TklW,DEFENSE_Tackles_Def_3rd,DEFENSE_Tackles_Mid_3rd,DEFENSE_Tackles_Att_3rd,DEFENSE_Challenges_Tkl,DEFENSE_Challenges_Att,DEFENSE_Challenges_Tkl%,DEFENSE_Challenges_Lost,DEFENSE_Blocks_Blocks,DEFENSE_Blocks_Sh,DEFENSE_Blocks_Pass,DEFENSE_Int,DEFENSE_Tkl+Int,DEFENSE_Clr,DEFENSE_Err,POSSESSION_90s,POSSESSION_Touches_Touches,POSSESSION_Touches_Def_Pen,POSSESSION_Touches_Def_3rd,POSSESSION_Touches_Mid_3rd,POSSESSION_Touches_Att_3rd,POSSESSION_Touches_Att_Pen,POSSESSION_Touches_Live,POSSESSION_Take-Ons_Att,POSSESSION_Take-Ons_Succ,POSSESSION_Take-Ons_Succ%,POSSESSION_Take-Ons_Tkld,POSSESSION_Take-Ons_Tkld%,POSSESSION_Carries_Carries,POSSESSION_Carries_TotDist,POSSESSION_Carries_PrgDist,POSSESSION_Carries_PrgC,POSSESSION_Carries_1/3,POSSESSION_Carries_CPA,POSSESSION_Carries_Mis,POSSESSION_Carries_Dis,POSSESSION_Receiving_Rec,POSSESSION_Receiving_PrgR,PLAYING_TIME_MP,PLAYING_TIME_Playing_Time_Min,PLAYING_TIME_Playing_Time_Mn/MP,PLAYING_TIME_Playing_Time_Min%,PLAYING_TIME_Playing_Time_90s,PLAYING_TIME_Starts_Starts,PLAYING_TIME_Starts_Mn/Start,PLAYING_TIME_Starts_Compl,PLAYING_TIME_Subs_Subs,PLAYING_TIME_Subs_Mn/Sub,PLAYING_TIME_Subs_unSub,PLAYING_TIME_Team_Success_PPM,PLAYING_TIME_Team_Success_onG,PLAYING_TIME_Team_Success_onGA,PLAYING_TIME_Team_Success_+/-,PLAYING_TIME_Team_Success_+/-90,PLAYING_TIME_Team_Success_On-Off,PLAYING_TIME_Team_Success_(xG)_onxG,PLAYING_TIME_Team_Success_(xG)_onxGA,PLAYING_TIME_Team_Success_(xG)_xG+/-,PLAYING_TIME_Team_Success_(xG)_xG+/-90,PLAYING_TIME_Team_Success_(xG)_On-Off,MISC_90s,MISC_Performance_CrdY,MISC_Performance_CrdR,MISC_Performance_2CrdY,MISC_Performance_Fls,MISC_Performance_Fld,MISC_Performance_Off,MISC_Performance_Crs,MISC_Performance_Int,MISC_Performance_TklW,MISC_Performance_PKwon,MISC_Performance_PKcon,MISC_Performance_OG,MISC_Performance_Recov,MISC_Aerial_Duels_Won,MISC_Aerial_Duels_Lost,MISC_Aerial_Duels_Won%


### National Team Stats

In [25]:
nat_tm

,FBRefId,Season,Squad,Comp,Age,LgRank,STANDARD_MP,STANDARD_Playing_Time_Starts,STANDARD_Playing_Time_Min,STANDARD_Playing_Time_90s,STANDARD_Performance_Gls,STANDARD_Performance_Ast,STANDARD_Performance_G+A,STANDARD_Performance_G-PK,STANDARD_Performance_PK,STANDARD_Performance_PKatt,STANDARD_Performance_CrdY,STANDARD_Performance_CrdR,STANDARD_Progression_PrgC,STANDARD_Progression_PrgP,STANDARD_Progression_PrgR,STANDARD_Per_90_Minutes_Gls,STANDARD_Per_90_Minutes_Ast,STANDARD_Per_90_Minutes_G+A,STANDARD_Per_90_Minutes_G-PK,STANDARD_Per_90_Minutes_G+A-PK,PASSING_90s,PASSING_Total_Cmp,PASSING_Total_Att,PASSING_Total_Cmp%,PASSING_Total_TotDist,PASSING_Total_PrgDist,PASSING_Short_Cmp,PASSING_Short_Att,PASSING_Short_Cmp%,PASSING_Medium_Cmp,PASSING_Medium_Att,PASSING_Medium_Cmp%,PASSING_Long_Cmp,PASSING_Long_Att,PASSING_Long_Cmp%,PASSING_Ast,PASSING_A-xAG,PASSING_KP,PASSING_1/3,PASSING_PPA,PASSING_CrsPA,PASSING_PrgP,PASSING_TYPES_90s,PASSING_TYPES_Att,PASSING_TYPES_Pass_Types_Live,PASSING_TYPES_Pass_Types_Dead,PASSING_TYPES_Pass_Types_FK,PASSING_TYPES_Pass_Types_TB,PASSING_TYPES_Pass_Types_Sw,PASSING_TYPES_Pass_Types_Crs,PASSING_TYPES_Pass_Types_TI,PASSING_TYPES_Pass_Types_CK,PASSING_TYPES_Corner_Kicks_In,PASSING_TYPES_Corner_Kicks_Out,PASSING_TYPES_Corner_Kicks_Str,PASSING_TYPES_Outcomes_Cmp,PASSING_TYPES_Outcomes_Off,PASSING_TYPES_Outcomes_Blocks,GCA_90s,GCA_SCA_SCA,GCA_SCA_SCA90,GCA_SCA_Types_PassLive,GCA_SCA_Types_PassDead,GCA_SCA_Types_TO,GCA_SCA_Types_Sh,GCA_SCA_Types_Fld,GCA_SCA_Types_Def,GCA_GCA_GCA,GCA_GCA_GCA90,GCA_GCA_Types_PassLive,GCA_GCA_Types_PassDead,GCA_GCA_Types_TO,GCA_GCA_Types_Sh,GCA_GCA_Types_Fld,GCA_GCA_Types_Def,DEFENSE_90s,DEFENSE_Tackles_Tkl,DEFENSE_Tackles_TklW,DEFENSE_Tackles_Def_3rd,DEFENSE_Tackles_Mid_3rd,DEFENSE_Tackles_Att_3rd,DEFENSE_Challenges_Tkl,DEFENSE_Challenges_Att,DEFENSE_Challenges_Tkl%,DEFENSE_Challenges_Lost,DEFENSE_Blocks_Blocks,DEFENSE_Blocks_Sh,DEFENSE_Blocks_Pass,DEFENSE_Int,DEFENSE_Tkl+Int,DEFENSE_Clr,DEFENSE_Err,POSSESSION_90s,POSSESSION_Touches_Touches,POSSESSION_Touches_Def_Pen,POSSESSION_Touches_Def_3rd,POSSESSION_Touches_Mid_3rd,POSSESSION_Touches_Att_3rd,POSSESSION_Touches_Att_Pen,POSSESSION_Touches_Live,POSSESSION_Take-Ons_Att,POSSESSION_Take-Ons_Succ,POSSESSION_Take-Ons_Succ%,POSSESSION_Take-Ons_Tkld,POSSESSION_Take-Ons_Tkld%,POSSESSION_Carries_Carries,POSSESSION_Carries_TotDist,POSSESSION_Carries_PrgDist,POSSESSION_Carries_PrgC,POSSESSION_Carries_1/3,POSSESSION_Carries_CPA,POSSESSION_Carries_Mis,POSSESSION_Carries_Dis,POSSESSION_Receiving_Rec,POSSESSION_Receiving_PrgR,PLAYING_TIME_MP,PLAYING_TIME_Playing_Time_Min,PLAYING_TIME_Playing_Time_Mn/MP,PLAYING_TIME_Playing_Time_Min%,PLAYING_TIME_Playing_Time_90s,PLAYING_TIME_Starts_Starts,PLAYING_TIME_Starts_Mn/Start,PLAYING_TIME_Starts_Compl,PLAYING_TIME_Subs_Subs,PLAYING_TIME_Subs_Mn/Sub,PLAYING_TIME_Subs_unSub,PLAYING_TIME_Team_Success_PPM,PLAYING_TIME_Team_Success_onG,PLAYING_TIME_Team_Success_onGA,PLAYING_TIME_Team_Success_+/-,PLAYING_TIME_Team_Success_+/-90,PLAYING_TIME_Team_Success_On-Off,MISC_90s,MISC_Performance_CrdY,MISC_Performance_CrdR,MISC_Performance_2CrdY,MISC_Performance_Fls,MISC_Performance_Fld,MISC_Performance_Off,MISC_Performance_Crs,MISC_Performance_Int,MISC_Performance_TklW,MISC_Performance_PKwon,MISC_Performance_PKcon,MISC_Performance_OG
0,466fb2c5,2021,eng England,Friendlies (M),22,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,466fb2c5,2021,eng England,UEFA Euro,22,F,0.0,0.0,NaN,NaN,NaN